###  Install and import libraries

In [ ]:
import pandas as pd
import numpy as np
import hashlib
import matplotlib.pyplot as plt 
import seaborn as sns

### Manage and hash data locally

In [ ]:
def hash_value(value):
    return hashlib.sha256(str(value).encode('utf-8')).hexdigest()

columns_to_hash = ['funding_masked_card_number', 'payment_masked_card_number',
                   'funding_card_bank_name', 'payment_card_bank_name']

for column in columns_to_hash:
    df[column] = df[column].apply(hash_value)

In [ ]:
# Specify the path and filename for the output CSV file
output_file = '../data/findo_2022_06-08_h.csv'

# Save the combined DataFrame to a CSV file
df.to_csv(output_file, index=False)

### Manage hashed data on remote dev machine

This code gets hashed Findo dataset, cleans it, gets around 100k samples, manages necessary columns transformations, etc. and saves dataset as S-FFSD.csv

In [ ]:
df = pd.read_csv('../data/sample_final_hashed_dataset.csv')

# Drop column 'correlation_id.1'
df = df.drop(columns=['correlation_id.1'])

# Filter rows where 'status' is either 'APPROVED' or 'DECLINED'
df = df[df['status'].isin(['APPROVED', 'DECLINED'])]

# Add a new column 'labels' with default value 0
df['labels'] = 0

# Replace 'labels' values with '1' for currencies 'GHS', 'NGN', 'TTD', 'GMD','MGA' in the 'labels' column:
df.loc[df['currency'].isin(['GHS', 'NGN', 'TTD', 'GMD', 'MGA']), 'labels'] = 1

# Ensure the DataFrame is sorted by 'created_date'
df.sort_values(by='created_date', inplace=True)

# Add column 'time' and number rows in 'time' column starting from 0 
df['time'] = range(len(df))

# Leave only the columns: 'time', 'amount_usd', 'currency', 'labels', 'funding_masked_card_number', 'payment_masked_card_number', 'payment_card_country', 'payment_transaction_status'
df = df[['time', 'amount_usd', 'labels', 'funding_masked_card_number', 'payment_masked_card_number', 'payment_card_country', 'status']]

# Rename columns 'time' -> 'Time', 'funding_masked_card_number' -> 'Source', 'payment_masked_card_number' -> 'Target', 'amount_usd' -> 'Amount', 'labels' -> 'Labels',
# 'payment_card_country' -> 'Location', 'status' -> 'Type'
df = df.rename(columns={
    'time': 'Time',
    'funding_masked_card_number': 'Source',
    'payment_masked_card_number': 'Target',
    'amount_usd': 'Amount',
    'labels': 'Labels',
    'payment_card_country': 'Location',
    'status': 'Type'
})

# Put columns in df in the following order: 'Time', 'Source', 'Target', 'Amount', 'Location', 'Type', 'Labels'
df = df[['Time', 'Source', 'Target', 'Amount', 'Location', 'Type', 'Labels']]

# Find the first and last row indices where 'Labels' equals 1
first_index = df[df['Labels'] == 1].index[0]
last_index = df[df['Labels'] == 1].index[-1]

# Create a new dataframe containing all rows between these indices
result_df = df.loc[first_index:last_index+1]

# Create a mask for rows with 'Labels' equal to 0
mask = result_df['Labels'] == 0

# Get the indices of 20% of the rows with 'Labels' equal to 0
indices_to_replace = result_df[mask].sample(frac=0.2).index

# Replace the 'Labels' values at the selected indices with 2
result_df.loc[indices_to_replace, 'Labels'] = 2

# Replace values in 'Time' column: fill with numbers starting from 0 to the amount of rows
result_df['Time'] = range(len(result_df))

# Save dataframe to csv
result_df.to_csv('../data/S-FFSD.csv', index=False)

In [ ]:
# This code loads the dataset and ensures that all columns and all data is in proper shape

# Load data/S-FFSD.csv to df (This is Findo data sampled from the whole dataset and prepared for transformation to be used by training script)
df = pd.read_csv('../data/S-FFSD.csv')

# Show the amount of rows with 'Labels' 0, 1 and 2
print("Amount of rows with 'Labels' 0:", df[df['Labels'] == 0].shape[0])
print("Amount of rows with 'Labels' 1:", df[df['Labels'] == 1].shape[0])
print("Amount of rows with 'Labels' 2:", df[df['Labels'] == 2].shape[0])

## Import libraries and methods

This to experiment in notebook before running the graph training script

In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np
from scipy.io import loadmat
import torch
import dgl
import random
import os
import time
import argparse
import pickle
import matplotlib.pyplot as plt
import networkx as nx
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

DATADIR = "../data/"

def featmap_gen(tmp_df=None):
    """
    Handle S-FFSD dataset and do some feature engineering
    :param tmp_df: the feature of input dataset
    """
    # time_span = [2, 5, 12, 20, 60, 120, 300, 600, 1500, 3600, 10800, 32400, 64800, 129600,
    #              259200]  # Increase in the number of time windows to increase the characteristics.
    time_span = [2, 3, 5, 15, 20, 50, 100, 150,
                 200, 300, 864, 2590, 5100, 10000, 24000]
    time_name = [str(i) for i in time_span]
    time_list = tmp_df['Time']
    post_fe = []
    for trans_idx, trans_feat in tqdm(tmp_df.iterrows()):
        new_df = pd.Series(trans_feat)
        temp_time = new_df.Time
        temp_amt = new_df.Amount
        for length, tname in zip(time_span, time_name):
            lowbound = (time_list >= temp_time - length)
            upbound = (time_list <= temp_time)
            correct_data = tmp_df[lowbound & upbound]
            new_df['trans_at_avg_{}'.format(
                tname)] = correct_data['Amount'].mean()
            new_df['trans_at_totl_{}'.format(
                tname)] = correct_data['Amount'].sum()
            new_df['trans_at_std_{}'.format(
                tname)] = correct_data['Amount'].std()
            new_df['trans_at_bias_{}'.format(
                tname)] = temp_amt - correct_data['Amount'].mean()
            new_df['trans_at_num_{}'.format(tname)] = len(correct_data)
            new_df['trans_target_num_{}'.format(tname)] = len(
                correct_data.Target.unique())
            new_df['trans_location_num_{}'.format(tname)] = len(
                correct_data.Location.unique())
            new_df['trans_type_num_{}'.format(tname)] = len(
                correct_data.Type.unique())
        post_fe.append(new_df)
    return pd.DataFrame(post_fe)


def sparse_to_adjlist(sp_matrix, filename):
    """
    Transfer sparse matrix to adjacency list
    :param sp_matrix: the sparse matrix
    :param filename: the filename of adjlist
    """
    # add self loop
    homo_adj = sp_matrix + sp.eye(sp_matrix.shape[0])
    # create adj_list
    adj_lists = defaultdict(set)
    edges = homo_adj.nonzero()
    for index, node in enumerate(edges[0]):
        adj_lists[node].add(edges[1][index])
        adj_lists[edges[1][index]].add(node)
    with open(filename, 'wb') as file:
        pickle.dump(adj_lists, file)
    file.close()


def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def MinMaxScaling(data):
    mind, maxd = data.min(), data.max()
    # return mind + (data - mind) / (maxd - mind)
    return (data - mind) / (maxd - mind)


def k_neighs(
    graph: dgl.DGLGraph,
    center_idx: int,
    k: int,
    where: str,
    choose_risk: bool = False,
    risk_label: int = 1
) -> torch.Tensor:
    """return indices of risk k-hop neighbors

    Args:
        graph (dgl.DGLGraph): dgl graph dataset
        center_idx (int): center node idx
        k (int): k-hop neighs
        where (str): {"predecessor", "successor"}
        risk_label (int, optional): value of fruad label. Defaults to 1.
    """
    target_idxs: torch.Tensor
    if k == 1:
        if where == "in":
            neigh_idxs = graph.predecessors(center_idx)
        elif where == "out":
            neigh_idxs = graph.successors(center_idx)

    elif k == 2:
        if where == "in":
            subg_in = dgl.khop_in_subgraph(
                graph, center_idx, 2, store_ids=True)[0]
            neigh_idxs = subg_in.ndata[dgl.NID][subg_in.ndata[dgl.NID] != center_idx]
            # delete center node itself
            neigh1s = graph.predecessors(center_idx)
            neigh_idxs = neigh_idxs[~torch.isin(neigh_idxs, neigh1s)]
        elif where == "out":
            subg_out = dgl.khop_out_subgraph(
                graph, center_idx, 2, store_ids=True)[0]
            neigh_idxs = subg_out.ndata[dgl.NID][subg_in.ndata[dgl.NID] != center_idx]
            neigh1s = graph.successors(center_idx)
            neigh_idxs = neigh_idxs[~torch.isin(neigh_idxs, neigh1s)]

    neigh_labels = graph.ndata['label'][neigh_idxs]
    if choose_risk:
        target_idxs = neigh_idxs[neigh_labels == risk_label]
    else:
        target_idxs = neigh_idxs

    return target_idxs


def count_risk_neighs(
    graph: dgl.DGLGraph,
    risk_label: int = 1
) -> torch.Tensor:

    ret = []
    for center_idx in graph.nodes():
        neigh_idxs = graph.successors(center_idx)
        neigh_labels = graph.ndata['label'][neigh_idxs]
        risk_neigh_num = (neigh_labels == risk_label).sum()
        ret.append(risk_neigh_num)

    return torch.Tensor(ret)


def feat_map():
    tensor_list = []
    feat_names = []
    for idx in tqdm(range(graph.num_nodes())):
        neighs_1_of_center = k_neighs(graph, idx, 1, "in")
        neighs_2_of_center = k_neighs(graph, idx, 2, "in")

        tensor = torch.FloatTensor([
            edge_feat[neighs_1_of_center, 0].sum().item(),
            # edge_feat[neighs_1_of_center, 0].std().item(),
            edge_feat[neighs_2_of_center, 0].sum().item(),
            # edge_feat[neighs_2_of_center, 0].std().item(),
            edge_feat[neighs_1_of_center, 1].sum().item(),
            # edge_feat[neighs_1_of_center, 1].std().item(),
            edge_feat[neighs_2_of_center, 1].sum().item(),
            # edge_feat[neighs_2_of_center, 1].std().item(),
        ])
        tensor_list.append(tensor)

    feat_names = ["1hop_degree", "2hop_degree",
                  "1hop_riskstat", "2hop_riskstat"]

    tensor_list = torch.stack(tensor_list)
    return tensor_list, feat_names

### Create Findo dataset

In [ ]:
set_seed(42)

# """
#     For S-FFSD dataset
# """
print(f"processing Findo data...")
data = pd.read_csv(os.path.join(DATADIR, 'findo_2022_06-08_h_fe.csv'))
data = featmap_gen(data.reset_index(drop=True))
data.replace(np.nan, 0, inplace=True)
data.to_csv(os.path.join(DATADIR, 'Findoneofull.csv'), index=None)
data = pd.read_csv(os.path.join(DATADIR, 'Findoneofull.csv'))

data = data.reset_index(drop=True)
out = []
alls = []
allt = []
pair = ["Source", "Target", "Location", "Type"]
for column in pair:
    src, tgt = [], []
    edge_per_trans = 3
    for c_id, c_df in tqdm(data.groupby(column), desc=column):
        c_df = c_df.sort_values(by="Time")
        df_len = len(c_df)
        sorted_idxs = c_df.index
        src.extend([sorted_idxs[i] for i in range(df_len)
                    for j in range(edge_per_trans) if i + j < df_len])
        tgt.extend([sorted_idxs[i+j] for i in range(df_len)
                    for j in range(edge_per_trans) if i + j < df_len])
    alls.extend(src)
    allt.extend(tgt)
alls = np.array(alls)
allt = np.array(allt)
g = dgl.graph((alls, allt))
cal_list = ["Source", "Target", "Location", "Type"]
for col in cal_list:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].apply(str).values)
feat_data = data.drop("Labels", axis=1)
labels = data["Labels"]
g.ndata['label'] = torch.from_numpy(
    labels.to_numpy()).to(torch.long)
g.ndata['feat'] = torch.from_numpy(
    feat_data.to_numpy()).to(torch.float32)
dgl.data.utils.save_graphs(DATADIR + "graph-Findo.bin", [g])

# generate neighbor riskstat features
for file_name in ['Findo']:
    print(
        f"Generating neighbor risk-aware features for {file_name} dataset...")
    graph = dgl.load_graphs(DATADIR + "graph-" + file_name + ".bin")[0][0]
    graph: dgl.DGLGraph
    print(f"graph info: {graph}")

    edge_feat: torch.Tensor
    degree_feat = graph.in_degrees().unsqueeze_(1).float()
    risk_feat = count_risk_neighs(graph).unsqueeze_(1).float()

    origin_feat_name = []
    edge_feat = torch.cat([degree_feat, risk_feat], dim=1)
    origin_feat_name = ['degree', 'riskstat']

    features_neigh, feat_names = feat_map()
    # print(f"feature neigh: {features_neigh.shape}")

    features_neigh = torch.cat(
        (edge_feat, features_neigh), dim=1
    ).numpy()
    feat_names = origin_feat_name + feat_names
    features_neigh[np.isnan(features_neigh)] = 0.

    output_path = DATADIR + file_name + "_neigh_feat.csv"
    features_neigh = pd.DataFrame(features_neigh, columns=feat_names)
    scaler = StandardScaler()
    # features_neigh = np.log(features_neigh + 1)
    features_neigh = pd.DataFrame(scaler.fit_transform(
        features_neigh), columns=features_neigh.columns)

    features_neigh.to_csv(output_path, index=False)

### Features correlation analisis

In [ ]:
# Drop columns to prepare for correlation analisis
columns_to_drop = ['created_date', 'correlation_id', 'mcc', 'transaction_type', 'payment_masked_card_number', 'funding_masked_card_number']
df = df.drop(columns=columns_to_drop)

df['payment_transaction_status'] = df['payment_transaction_status'].fillna('0')

In [ ]:
# Convert categorical columns to numerical values if necessary
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['class', 'status', 'currency', 'funding_provider', 'payment_provider', 'payment_card_country', 'funding_card_country', 'funding_card_bank_name', 'payment_transaction_status', 'payment_card_bank_name', 'acq_response_code', 'payment_transaction_currency']

label_encoder = LabelEncoder()
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column].astype(str))

df = df.fillna(0)
df['amount_usd'] = pd.to_numeric(df['amount_usd'], errors='coerce')
df = df.reset_index(drop=True) 


In [ ]:
print("Unique currency codes or names : ", df['currency'].unique())

In [ ]:
# Create the Correlation Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', square=True)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()